# Graficos

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as st
from scipy.stats import linregress
import csv

In [ ]:
dataframe = pd.read_csv('output3.csv', sep=';')

tiempos_promedio_nx = []
tiempos_promedio_ht = []
tiempos_promedio_str = []
valor_inicial = 100

for i in range(13):
    tiempos_promedio_nx.append(float(dataframe.iloc[valor_inicial, 1])/100)
    tiempos_promedio_ht.append(float(dataframe.iloc[valor_inicial, 3])/100)
    tiempos_promedio_str.append(float(dataframe.iloc[valor_inicial, 5])/100)
    valor_inicial += 102

etiquetas_x = [f'2^{i}' for i in range(10, 23)]

plt.figure(figsize=(16, 26))
plt.plot(etiquetas_x, tiempos_promedio_nx, marker='o', color='b', linestyle='-', label='Método 1: Nearest X')
plt.plot(etiquetas_x, tiempos_promedio_ht, marker='o', color='r', linestyle=':', label='Método 2: Hilbert R-tree')
plt.plot(etiquetas_x, tiempos_promedio_str, marker='o', color='k', linestyle='--', label='Método 3: STR')
plt.legend()
plt.xlabel('Tamaño de entrada de los datos')
plt.ylabel('Tiempo de búsqueda promedio (µs)')
plt.title('Tiempos de búsqueda promedio de rectángulos en función del tamaño de entrada')
plt.xticks(rotation=45)
plt.tight_layout()



plt.savefig('resultados/grafico_tiempos_promedio.pdf')
plt.show()

In [ ]:
ios_promedio_nx = []
ios_promedio_ht = []
ios_promedio_str = []
valor_columna = 100

for i in range(16):
    ios_promedio_nx.append(int(dataframe.iloc[valor_columna, 2])/100)
    ios_promedio_ht.append(int(dataframe.iloc[valor_columna, 4])/100)
    ios_promedio_str.append(int(dataframe.iloc[valor_columna, 6])/100)
    valor_columna += 102

etiquetas_x = [f'2^{i}' for i in range(10, 26)]

plt.figure(figsize=(10, 6))
plt.plot(etiquetas_x, ios_promedio_nx, marker='o', color='b', linestyle='-', label='Método 1: Nearest X')
plt.plot(etiquetas_x, ios_promedio_ht, marker='o', color='r', linestyle=':', label='Método 2: Hilbert R-tree')
plt.plot(etiquetas_x, ios_promedio_str, marker='o', color='k', linestyle='--', label='Método 3: STR')
plt.legend()
plt.xlabel('Tamaño de entrada de los datos')
plt.ylabel('I/Os promedio de la consulta')
plt.title('Cantidad de accesos promedio a bloques de disco de la búsqueda en función del tamaño de entrada')


plt.xticks(rotation=45)
plt.tight_layout()


plt.savefig('resultados/grafico_ios.pdf')
plt.show()